# YOLOv4

In [ ]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNet("C:/Users/ADMIN/Downloads/cross-hands.weights", "C:/Users/ADMIN/Downloads/cross-hands.cfg")

# Load video capture with reduced resolution
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set the width of the frame
cap.set(4, 480)  # Set the height of the frame

# Create a window to display the cropped hand
cv2.namedWindow("Cropped Hand", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Cropped Hand", 300, 300)  # Adjust the size as needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the input blob for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Lists to store detected hands
    hand_boxes = []

    height, width, _ = frame.shape

    # Loop over each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.3 and class_id == 0:  # Adjust the threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Get the top-left coordinates of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                hand_boxes.append((x, y, w, h))

                # Crop the detected hand and display it in another frame
                if len(hand_boxes) > 0:
                    x, y, w, h = hand_boxes[0]
                    cropped_hand = frame[y:y+h, x:x+w]
                    cv2.imshow("Cropped Hand", cropped_hand)

                # Draw bounding box and label for the first detected hand
                x, y, w, h = hand_boxes[0]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, "Hand", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Break the loop after the first detected hand
                break

    # Show the frame with detected hand
    cv2.imshow("Hand Detection", frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()

# CNN + YOLOv4

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load YOLO model
net = cv2.dnn.readNet("D:/Drone/cross-hands-yolov4-tiny.weights", "D:/Drone/cross-hands-yolov4-tiny.cfg")

# Load the hand gesture recognition model
hand_gesture_model = tf.keras.models.load_model("hand_gesture_model01.h5")

# Define class labels
class_labels = ["down", "up", "right", "back", "front", "left"]

# Load video capture with reduced resolution
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set the width of the frame
cap.set(4, 480)  # Set the height of the frame

# Create a window to display the cropped hand and predicted label
cv2.namedWindow("Cropped Hand", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Cropped Hand", 300, 300)  # Adjust the size as needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the input blob for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Lists to store detected hands
    hand_boxes = []

    height, width, _ = frame.shape

    # Loop over each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.3 and class_id == 0:  # Adjust the threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Get the top-left coordinates of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                hand_boxes.append((x, y, w, h))

                # Crop the detected hand
                if len(hand_boxes) > 0:
                    x, y, w, h = hand_boxes[0]
                    cropped_hand = frame[y:y+h, x:x+w]

                    # Preprocess the cropped hand image for the model
                    cropped_hand = cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB)
                    cropped_hand = cv2.resize(cropped_hand, (224, 224))  # Adjust the size as needed
                    cropped_hand = cropped_hand / 255.0  # Normalize pixel values

                    # Make a prediction using the hand gesture recognition model
                    predicted_label_idx = np.argmax(hand_gesture_model.predict(np.expand_dims(cropped_hand, axis=0))[0])
                    predicted_label = class_labels[predicted_label_idx]

                    # Show the predicted label on the screen
                    label_text = f"Predicted: {predicted_label}"
                    cv2.putText(frame, label_text, (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Draw bounding box for the detected hand
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Break the loop after the first detected hand
                break

    # Show the frame with detected hand and predicted label
    cv2.imshow("Hand Detection", frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()